# Piano PLayer 🎹

In [59]:
import cv2
import os

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from threading import Thread

from playsound import playsound

import math

### List of Sounds

In [60]:
Sounds = os.listdir("../sounds")

### Functions

In [61]:
def calculate_distance(ponto1, ponto2):
    return math.sqrt((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2)

### Piano

In [62]:
class PianoPlayer:
  def __init__(self):
    self.audio_executed = False
    
  def read_keys(self):
    for x in range(1, 5):
      cv2.circle(frame,(0+(x*80),50), 5, (0,0,255), -1)
    
  def play_sound(self, x):
    self.audio_executed = True
    playsound(f'C:/Users/andre/OneDrive/Documentos/piano/sounds/{Sounds[x]}')
    self.audio_executed = False
  
  def key_press(self, indicador_x, indicador_y):
    for x in range(1, 5):
      if calculate_distance((indicador_x, indicador_y), (0+(x*80),50)) <= 5 and not self.audio_executed:
        self.audio_executed = True
        self.play_sound(x)
      elif calculate_distance((indicador_x, indicador_y), (0+(x*80),50)) > 50:
        self.audio_executed = False

In [63]:
piano = PianoPlayer()

### Configure Hands

In [64]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

### CV2 

In [65]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
  
  ret, frame = cap.read()
  
  if not ret:
      print("Erro ao capturar o frame.")
      break
    
  # Read Keys
  piano.read_keys()
  
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  results = hands.process(rgb_frame)

  if results.multi_hand_landmarks and results.multi_handedness:
    
    for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
      
      hand_label = hand_handedness.classification[0].label
      
      h, w, _ = frame.shape
      
      # Point hand
      indicador = hand_landmarks.landmark[8]
      indicador_x, indicador_y = int(indicador.x * w), int(indicador.y * h)

      # Piano Press Key ?
      piano.key_press(indicador_x, indicador_y)
      
      mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
      
  cv2.imshow("MediaPipe Hands - Gestos Especificos", frame) 
  
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

## Functions

In [66]:
# def calculate_distance(ponto1, ponto2):
#     return math.sqrt((ponto1[0] - ponto2[0])**2 + (ponto1[1] - ponto2[1])**2)

In [67]:
# def Rules(distance):
#   global audio_executed
#   if distance < 5 and not audio_executed:
#     audio_executed = True
#     playsound('C:/Users/andre/OneDrive/Documentos/piano/sounds/0_G1.mp3')
#     audio_executed = False
#   elif distance > 50:
#     audio_executed = False

In [68]:
# def Keys():
#   p1 = cv2.circle(frame,(80,50), 5, (0,0,255), -1)
  
#   p2 = cv2.circle(frame,(160,50), 5, (0,0,255), -1)
  
#   p3 = cv2.circle(frame,(240,50), 5, (0,0,255), -1)
  
#   p4 = cv2.circle(frame,(320,50), 5, (0,0,255), -1)
  
#   p5 = cv2.circle(frame,(400,50), 5, (0,0,255), -1)

## Configure Hands

In [69]:
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(static_image_mode=False,
#                        max_num_hands=2,
#                        min_detection_confidence=0.5,
#                        min_tracking_confidence=0.5)

# mp_drawing = mp.solutions.drawing_utils

In [70]:
# cap = cv2.VideoCapture(0)

# while cap.isOpened():
  
#   ret, frame = cap.read()
  
#   if not ret:
#       print("Erro ao capturar o frame.")
#       break
    
#   # Points
#   Keys()
  
#   rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#   results = hands.process(rgb_frame)

#   if results.multi_hand_landmarks and results.multi_handedness:
    
#     for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
      
#       hand_label = hand_handedness.classification[0].label
      
#       h, w, _ = frame.shape
      
#       # Point hand
#       indicador_8 = hand_landmarks.landmark[8]
#       indicador_8_x, indicador_8_y = int(indicador_8.x * w), int(indicador_8.y * h)
      
#       distance = calculate_distance(
#         (indicador_8_x, indicador_8_y),
#         (80, 50)
#       )
      
#       # Sounds
      
#       # Rules 
#       rules = Thread(target=Rules, args=(distance,), daemon=True)
#       rules.start()
      
#       mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
      
#   cv2.imshow("MediaPipe Hands - Gestos Especificos", frame) 
  
#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break

# cap.release()
# cv2.destroyAllWindows()